In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
os.chdir('D:/Destop/')
         #  cd
# ?pd.read_excel
df =pd.read_excel('豆瓣电影数据.xlsx',sheet_name=0,header=0)
print('数据长度',len(df))
print('数据字段为',df.columns.tolist())

数据长度 2
数据字段为 ['name', '导演', '主演']


In [3]:
# 数据清洗
data = df[['name','导演','主演']]
data.dropna(inplace=True)

data_yy = data['主演'].str.split('/',expand=True)   # 将这一列生成一个全新的EXCEL
print(data)
print('----')
col_len1=len(data_yy.columns)
data_yy.columns = ['yy'+str(i) for i in range(col_len1)]
print(data_yy.head())

print('----')
data_dy = data['导演'].str.split('/',expand=True)   # 将这一列生成一个全新的EXCEL
col_len2=len(data_dy.columns)
data_dy.columns = ['dy'+str(i) for i in range(col_len2)]
print(data_dy.head())


data2 = data_dy.join(data_yy).join(data['name'])   # 一行一行地合并
print(data2)
data2[data2['yy3'].notnull()]

  name     导演       主演
0   张三  A/B/C  D/E/F/G
1   张s    A/B    D/E/F
----
  yy0 yy1 yy2   yy3
0   D   E   F     G
1   D   E   F  None
----
  dy0 dy1   dy2
0   A   B     C
1   A   B  None
  dy0 dy1   dy2 yy0 yy1 yy2   yy3 name
0   A   B     C   D   E   F     G   张三
1   A   B  None   D   E   F  None   张s


,dy0,dy1,dy2,yy0,yy1,yy2,yy3,name
0,A,B,C,D,E,F,G,张三


In [12]:
# 拆分 + 合并
data_re = pd.DataFrame(columns=['name','导演','演员'])
col_yy = data_yy.columns
col_dy = data_dy.columns

for dy in col_dy:
    for yy in col_yy:
        data_i = data2[['name',dy,yy]].dropna()  # 输出是同一个dy,yy的电影组合,有一个为none去掉
        data_i.columns=['name','导演','演员']
#         print(data_i)
        data_re = pd.concat([data_re,data_i])# 上下合并
    
data_re.reset_index(inplace=True,drop=True)
print(data_re)


   name 导演 演员
0    张三  A  D
1    张s  A  D
2    张三  A  E
3    张s  A  E
4    张三  A  F
5    张s  A  F
6    张三  A  G
7    张三  B  D
8    张s  B  D
9    张三  B  E
10   张s  B  E
11   张三  B  F
12   张s  B  F
13   张三  B  G
14   张三  C  D
15   张三  C  E
16   张三  C  F
17   张三  C  G


In [16]:
# 汇总统计导演和演员的合作次数
result = data_re.groupby(['导演','演员']).count()
print(result)
# result.reset_index(inplace=True,drop=True)

# 按照导演--->演员进行计数统计，得到结果数据
# reset_index  将所有索引级别转换为列

writer = pd.ExcelWriter('output.xlsx')
result.to_excel(writer,'sheet1')

writer.save()

       name
导演 演员      
A  D      2
   E      2
   F      2
   G      1
B  D      2
   E      2
   F      2
   G      1
C  D      1
   E      1
   F      1
   G      1
